## Upload personal data to database
## ( the best version )

#### Import the libraries to get the env variables

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
OPENAI_API_VERSION = "2023-03-15-preview"

In [3]:
OPENAI_API_KEY 

'9ac347d13e834f288a2076ff9c7b418a'

#### Set the name of the database

In [4]:
database = "./barack_obama_documents/"

#### Import the libraries to get the text and split it in chunks

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import TextLoader, DirectoryLoader, PyPDFLoader

##### Get the text from the documents

In [6]:
# loader = DirectoryLoader(database, glob="./*.txt", loader_cls=TextLoader)
loader = DirectoryLoader(database, glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()
documents


[Document(page_content='On July 27, 2004, Illinois State Senator Barack Obama delivered the keynote speech at the Democratic \nNational Convention. He said, “Tonight is a particular honor for me because, let’s face it, my presence on \nthis stage is pretty unlikely.” When he finished his speech, the audience that listened with rapt attention \nexcitedly waved their arms, hats, and signs, thrilled with what they had just heard. Afterward, those \nwatching on television said that they had stood and cheered, many admitting they dance d. Some \nwondered what had just happened. For many Democrats, the speech was electrifying and inspiring; for \nthem, it was a joyful time. And those from the other side of the political aisle who watched and listened \nhad to agree: this fresh face, this politi co, unknown to nearly everyone in the country outside of his \nhome state of Illinois, had just delivered a remarkable speech. Many asked, who is this man and where \ndid he come from? They asked why 

##### Split the text into chunks

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
docs = text_splitter.split_documents(documents)

len(docs)

36

#### Import the libraries to connect to Chroma database

In [8]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [9]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


##### Upload the text in chroma database

In [10]:
import re

text = "barack_obama_documents"
result = re.search(r'(?P<word>\w+)_', text)

if result:
    extracted_word = result.group('word')
    print(extracted_word)

database_name = extracted_word+"-database"
database_name

barack_obama


'barack_obama-database'

In [11]:
db2 = Chroma.from_documents(docs, embedding_function, persist_directory=database_name)